## Clonando repositorio e Importando bibliotecas e dados

In [1]:
!git clone https://github.com/ufrpe-machine-learning-2023/projeto-ml-2023.git

Cloning into 'projeto-ml-2023'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 72 (delta 12), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (72/72), 11.14 MiB | 7.86 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [2]:
import pandas as pd
from numpy import mean
import time
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn import neighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder

In [3]:
result_df = pd.read_csv("/content/projeto-ml-2023/dados/dados_limpos.csv")
result_df

,level_0,index,ano,regiao,unidade_da_federacao,municipio,codigo_municipio,total,total_f,feminino-1,...,feminino-6,feminino-7,total_m,masculino-1,masculino-2,masculino-3,masculino-4,masculino-5,masculino-6,masculino-7
0,0,12,2007,Norte,Rondônia,Alta Floresta D'Oeste,1100015,72,36,1,...,0,0,36,1,7,12,15,1,0,0
1,1,13,2007,Norte,Rondônia,Alto Alegre dos Parecis,1100379,22,13,0,...,0,0,9,1,1,3,4,0,0,0
2,2,14,2007,Norte,Rondônia,Alto Paraíso,1100403,28,20,1,...,0,0,8,1,0,5,2,0,0,0
3,3,15,2007,Norte,Rondônia,Alvorada D'Oeste,1100346,30,17,0,...,0,0,13,1,4,3,4,1,0,0
4,4,16,2007,Norte,Rondônia,Ariquemes,1100023,131,94,1,...,2,1,37,0,3,14,17,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89084,89084,5605,2022,Centro-Oeste,Goiás,Varjão,5221908,11,9,0,...,0,0,2,0,1,0,1,0,0,0
89085,89085,5606,2022,Centro-Oeste,Goiás,Vianópolis,5222005,27,17,1,...,0,0,10,2,3,2,1,0,2,0
89086,89086,5607,2022,Centro-Oeste,Goiás,Vicentinópolis,5222054,22,14,0,...,3,1,8,0,0,4,1,0,2,1
89087,89087,5608,2022,Centro-Oeste,Goiás,Vila Boa,5222203,8,8,0,...,0,0,0,0,0,0,0,0,0,0


## Configurando dados para treino

In [4]:
# Transformando dados que estavam em formato Wide em Long
df1 = result_df.iloc[:, 2:24]
df1.columns = ["ano" ,"regiao", "unidade_da_federacao", "municipio", "codigo_municipio", "total", "total_f", "<=24-f",
                  "25-29-f", "30-39-f", "40-49-f", "50-54-f", "55-59-f", ">=60-f", "total_m",
                         "<=24-m", "25-29-m", "30-39-m", "40-49-m", "50-54-m", "55-59-m", ">=60-m"]
df2 = pd.melt(df1, id_vars=["ano" ,"regiao", "unidade_da_federacao", "municipio", "codigo_municipio", "total", "total_f", "total_m"],
              var_name= "faixa_etaria", value_name="qtd")

dados = df2
dados['genero'] = dados['faixa_etaria'].str[-1]
dados['faixa_etaria'] = dados['faixa_etaria'].str[:-2]

dados['genero'] = dados['genero'].str.upper()

dados = dados.pivot(index=["ano" ,"regiao", "unidade_da_federacao", "municipio", "codigo_municipio", "total", "total_f", "total_m", "genero"],
                    columns= 'faixa_etaria', values="qtd")

dados = dados.reset_index()
dados.columns.name = None
df = dados
df.iloc[:,9:] = df.iloc[:,9:].astype(int)


### Com Label encoder para transformar itens categoricos

In [5]:
# Separando dados de 2022 para validação com dados reais
df1 = df[df['ano'] == 2022]

# Retirando dados de 2022 do dataframe de treino
df = df[df['ano'] != 2022]

# Selecionando colunas que vão ser usadas como entrada
collunsUsed = ['ano', 'regiao', 'unidade_da_federacao', 'municipio', 'codigo_municipio', 'total', 'genero']
# collunsUsed = ['ano', 'total', 'regiao', 'genero']

# Selecionando colunas de String para conversão
collunsSTR = ['ano', 'regiao', 'unidade_da_federacao', 'municipio', 'codigo_municipio', 'genero']
# collunsSTR = ['ano', 'regiao', 'genero']

# Função de transformar categorias de Strings em numeros
le = preprocessing.LabelEncoder()

# Transformando colunas de Strings em numeros
df[collunsSTR] = df[collunsSTR].apply(le.fit_transform)
df1[collunsSTR] = df1[collunsSTR].apply(le.fit_transform)

# Escolha de coluna a ser predita
collunChoice = ['25-29', '30-39', '40-49', '50-54', '55-59', '<=24', '>=60']

# Selecionando conjunto de dados de Treino para Teste
X = df[collunsUsed]
y = df[collunChoice]

# Selecionando conjunto de dados Reais para validação
X_real = df1[collunsUsed]
y_real = df1[collunChoice]

<ipython-input-5-c8ac6dc97766>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[collunsSTR] = df[collunsSTR].apply(le.fit_transform)
<ipython-input-5-c8ac6dc97766>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[collunsSTR] = df1[collunsSTR].apply(le.fit_transform)


### Com GetDummies para transformar itens categoricos

In [38]:
# Filtrando colunas
collunsSelected = ['ano', 'total', 'regiao', 'genero', '25-29', '30-39', '40-49', '50-54', '55-59', '<=24', '>=60']


# Selecionando colunas que vão ser usadas como entrada
# collunsUsed = ['ano', 'regiao', 'unidade_da_federacao', 'municipio', 'codigo_municipio', 'total', 'genero']
collunsUsed = ['ano', 'total', 'regiao', 'genero']

# Selecionando colunas de String para conversão
# collunsSTR = ['ano', 'regiao', 'unidade_da_federacao', 'municipio', 'codigo_municipio', 'genero']
collunsSTR = ['ano', 'regiao', 'genero']

# Escolha de coluna a ser predita
collunChoice = ['25-29', '30-39', '40-49', '50-54', '55-59', '<=24', '>=60']

# # Transformando colunas de categoricas em numericas
dados = pd.get_dummies(df[collunsSelected], columns = collunsSTR)

# Separando dados de 2022 para validação com dados reais
dados_real = dados[dados['ano_2022'] == 1]

# Retirando dados de 2022 do dataframe de treino
dados_teste = dados[dados['ano_2022'] != 1]

# Selecionando conjunto de dados de Treino para Teste
y = dados_teste[collunChoice]
X = dados_teste.drop(collunChoice, axis=1)

# Selecionando conjunto de dados Reais para validação
y_real = dados_real[collunChoice]
X_real = dados_real.drop(collunChoice, axis=1)

In [39]:
X

,total,ano_2007,ano_2008,ano_2009,ano_2010,ano_2011,ano_2012,ano_2013,ano_2014,ano_2015,...,ano_2020,ano_2021,ano_2022,regiao_Centro-Oeste,regiao_Nordeste,regiao_Norte,regiao_Sudeste,regiao_Sul,genero_F,genero_M
0,3990,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,3990,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,20,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
3,20,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,31,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167035,61,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,1
167036,123,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,1,0
167037,123,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,1
167038,43,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,1,0


### Matrix de correlação

In [ ]:
# matriz de correlação de pearson
corr = sns.heatmap(X.corr(), annot=True, vmin=-1, vmax=1)
plt.show()

## Configurando modelos e variaveis

In [6]:
# Definindo o número de folds e repetições
num_folds = 10
num_repeats = 1

# Configurando inicializador de regressor linear
regressor = LinearRegression()

# Configurando inicializador de rede neural artificial
mlp = MLPRegressor(hidden_layer_sizes=(20,20,20), max_iter=1000, activation='relu', learning_rate='constant', early_stopping = True,
                     verbose= False, tol= 0.000001, n_iter_no_change= 20)

# Configurando inicializadores de KNN para distancias de 7
knn = neighbors.KNeighborsRegressor(n_neighbors = 7, weights= 'distance')

# Configurando inicializador de random forest
rf = RandomForestRegressor(max_depth=5, random_state=0)

# Configurando inicializador de regressor linear
dt = DecisionTreeRegressor()

# Dados de teste
# Criando Listas para pegar valores médidos de MSE, RMSE, MAE, R-Score e Tempo da Regressão Linear
RL_mse_teste = []
RL_rmse_teste = []
RL_mae_teste = []
RL_score_teste = []
RL_tempo = []

# Criando Listas para pegar valores médidos de MSE, RMSE, MAE, R-Score e Tempo da Rede Neural Artificial
RNA_mse_teste = []
RNA_rmse_teste = []
RNA_mae_teste = []
RNA_score_teste = []
RNA_tempo = []

# Criando Listas para pegar valores médidos de MSE, RMSE, MAE, R-Score e Tempo do KNN com k = 7
KNN_mse_teste = []
KNN_rmse_teste = []
KNN_mae_teste = []
KNN_score_teste = []
KNN_tempo = []

# Criando Listas para pegar valores médidos de MSE, RMSE, MAE, R-Score e Tempo da random forest
RF_mse_teste = []
RF_rmse_teste = []
RF_mae_teste = []
RF_score_teste = []
RF_tempo = []

# Criando Listas para pegar valores médidos de MSE, RMSE, MAE, R-Score e Tempo da arvore de decisão
DT_mse_teste = []
DT_rmse_teste = []
DT_mae_teste = []
DT_score_teste = []
DT_tempo = []

# Dados reais
# Criando Listas para pegar valores médidos de MSE, RMSE, MAE, R-Score e Tempo da Regressão Linear
RL_mse_real = []
RL_rmse_real = []
RL_mae_real = []
RL_score_real = []

# Criando Listas para pegar valores médidos de MSE, RMSE, MAE, R-Score e Tempo da Rede Neural Artificial
RNA_mse_real = []
RNA_rmse_real = []
RNA_mae_real = []
RNA_score_real = []

# Criando Listas para pegar valores médidos de MSE, RMSE, MAE, R-Score e Tempo do KNN com k = 7
KNN_mse_real = []
KNN_rmse_real = []
KNN_mae_real = []
KNN_score_real = []

# Criando Listas para pegar valores médidos de MSE, RMSE, MAE, R-Score e Tempo da random forest
RF_mse_real = []
RF_rmse_real = []
RF_mae_real = []
RF_score_real = []

# Criando Listas para pegar valores médidos de MSE, RMSE, MAE, R-Score e Tempo da arvore de decisão
DT_mse_real = []
DT_rmse_real = []
DT_mae_real = []
DT_score_real = []



In [7]:
# Looping para repetir o processo de validação cruzada 5 vezes
for repeat in range(num_repeats):
    print(f"Repetition {repeat + 1}")

    # Criando objetos KFold para dividir os dados
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=repeat)

    # Inicializando Timer para RL
    inicio = time.time()

    # Loop para executar a validação cruzada na Regressão Linear
    for fold, (train_index, test_index) in enumerate(kf.split(X)):
        print(f"Fold {fold + 1}")

        # Separando casos de Treino e casos de Teste
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Pre-processando
        scaler = StandardScaler()
        scaler.fit(X_train)

        # Normalizando dados de Teste
        X_train_scaled = scaler.transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Normalizando dados Reais
        X_real_scaled = scaler.transform(X_real)

        # Treinando modelo de RL baseado no X e y de Treino
        regressor.fit(X_train_scaled, y_train)

        # Realizando Predições
        y_pred = regressor.predict(X_test_scaled)
        ypred_real = regressor.predict(X_real_scaled)

        # Validando casos de Teste
        MSE = mean_squared_error(y_test, y_pred, squared=True)
        RMSE = mean_squared_error(y_test, y_pred, squared=False)
        MAE = mean_absolute_error(y_test, y_pred)
        cscore = r2_score(y_test, y_pred)

        # Validando casos Reais
        MSE_real = mean_squared_error(y_real, ypred_real, squared=True)
        RMSE_real = mean_squared_error(y_real, ypred_real, squared=False)
        MAE_real = mean_absolute_error(y_real, ypred_real)
        cscore_real = r2_score(y_real, ypred_real)

        # Adicionando valores a lista criada anteriormente para poder pegar a média
        RL_mse_teste.append(MSE)
        RL_rmse_teste.append(RMSE)
        RL_mae_teste.append(MAE)
        RL_score_teste.append(cscore)

        RL_mse_real.append(MSE_real)
        RL_rmse_real.append(RMSE_real)
        RL_mae_real.append(MAE_real)
        RL_score_real.append(cscore_real)

        # print dos valores preditos e reais para teste
        # teste1 = pd.DataFrame({"real": y_real, "predito": ypred_real.tolist()})
        # print(teste1.head(20))

    # Finalizando Timer e Adicionando a Lista de Tempo da RL
    fim = time.time()
    RL_tempo.append((fim - inicio))

    # Inicializando Timer para RNA
    inicio = time.time()

    # Loop para executar a validação cruzada da rede neural artificial
    for fold, (train_index, test_index) in enumerate(kf.split(X)):
        print(f"Fold {fold + 1}")

        # Separando casos de Treino e casos de Teste
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Pre-processando
        scaler = StandardScaler()
        scaler.fit(X_train)

        # Normalizando dados de Teste
        X_train_scaled = scaler.transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Normalizando dados Reais
        X_real_scaled = scaler.transform(X_real)

        # Treinando modelo de RNA baseado no X e y de Treino
        reg = mlp.fit(X_train_scaled, y_train)

        # Realizando Predições
        predictions = reg.predict(X_test_scaled)
        ypred_real = reg.predict(X_real_scaled)

        # Validando casos de Teste
        MSE = mean_squared_error(y_test, predictions, squared=True)
        RMSE = mean_squared_error(y_test, predictions, squared=False)
        MAE = mean_absolute_error(y_test, predictions)
        cscore = r2_score(y_test, predictions)

        # Validando casos Reais
        MSE_real = mean_squared_error(y_real, ypred_real, squared=True)
        RMSE_real = mean_squared_error(y_real, ypred_real, squared=False)
        MAE_real = mean_absolute_error(y_real, ypred_real)
        cscore_real = r2_score(y_real, ypred_real)

        # Adicionando valores a lista criada anteriormente para poder pegar a média
        RNA_mse_teste.append(MSE)
        RNA_rmse_teste.append(RMSE)
        RNA_mae_teste.append(MAE)
        RNA_score_teste.append(cscore)

        RNA_mse_real.append(MSE_real)
        RNA_rmse_real.append(RMSE_real)
        RNA_mae_real.append(MAE_real)
        RNA_score_real.append(cscore_real)

        # print dos valores preditos e reais para teste
        # teste2 = pd.DataFrame({"real": y_real, "predito": predictions_real.tolist()})
        # print(teste2.head(20))

    # Finalizando Timer e Adicionando a Lista de Tempo do RNA
    fim = time.time()
    RNA_tempo.append((fim - inicio))

    # Inicializando Timer para KNN com K = 1
    inicio = time.time()

    # Loop para executar a validação cruzada da KNN com k = 1
    for fold, (train_index, test_index) in enumerate(kf.split(X)):
        print(f"Fold {fold + 1}")

        # Separando casos de Treino e casos de Teste
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Pre-processando
        scaler = StandardScaler()
        scaler.fit(X_train)

        # Normalizando dados de Teste
        X_train_scaled = scaler.transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Normalizando dados Reais
        X_real_scaled = scaler.transform(X_real)

        # Treinando modelo de KNN baseado no X e y de Treino
        knn.fit(X_train_scaled, y_train)

        # Realizando Predições
        pred = knn.predict(X_test_scaled)
        ypred_real = knn.predict(X_real_scaled)

        # Validando casos de Teste
        MSE = mean_squared_error(y_test, pred, squared=True)
        RMSE = mean_squared_error(y_test, pred, squared=False)
        MAE = mean_absolute_error(y_test, pred)
        cscore = r2_score(y_test, pred)

        # Validando casos Reais
        MSE_real = mean_squared_error(y_real, ypred_real, squared=True)
        RMSE_real = mean_squared_error(y_real, ypred_real, squared=False)
        MAE_real = mean_absolute_error(y_real, ypred_real)
        cscore_real = r2_score(y_real, ypred_real)

        # Adicionando valores a lista criada anteriormente para poder pegar a média
        KNN_mse_teste.append(MSE)
        KNN_rmse_teste.append(RMSE)
        KNN_mae_teste.append(MAE)
        KNN_score_teste.append(cscore)

        KNN_mse_real.append(MSE_real)
        KNN_rmse_real.append(RMSE_real)
        KNN_mae_real.append(MAE_real)
        KNN_score_real.append(cscore_real)

        # print dos valores preditos e reais para teste
        # teste3 = pd.DataFrame({"real": y_real, "predito": pred_real.tolist()})
        # print(teste3.head(20))

    # Finalizando Timer e Adicionando a Lista de Tempo do KNN com k = 7
    fim = time.time()
    KNN_tempo.append((fim - inicio))

    # Inicializando Timer para random forest
    inicio = time.time()

    # Loop para executar a validação cruzada da random forest
    for fold, (train_index, test_index) in enumerate(kf.split(X)):
        print(f"Fold {fold + 1}")

        # Separando casos de Treino e casos de Teste
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Pre-processando
        scaler = StandardScaler()
        scaler.fit(X_train)

        # Normalizando dados de Teste
        X_train_scaled = scaler.transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Normalizando dados Reais
        X_real_scaled = scaler.transform(X_real)

        # Treinando modelo de random forest baseado no X e y de Treino
        rf.fit(X_train_scaled, y_train)

        # Realizando Predições
        pred = rf.predict(X_test_scaled)
        ypred_real = rf.predict(X_real_scaled)

        # Validando casos de Teste
        MSE = mean_squared_error(y_test, pred, squared=True)
        RMSE = mean_squared_error(y_test, pred, squared=False)
        MAE = mean_absolute_error(y_test, pred)
        cscore = r2_score(y_test, pred)

        # Validando casos Reais
        MSE_real = mean_squared_error(y_real, ypred_real, squared=True)
        RMSE_real = mean_squared_error(y_real, ypred_real, squared=False)
        MAE_real = mean_absolute_error(y_real, ypred_real)
        cscore_real = r2_score(y_real, ypred_real)

        # Adicionando valores a lista criada anteriormente para poder pegar a média
        RF_mse_teste.append(MSE)
        RF_rmse_teste.append(RMSE)
        RF_mae_teste.append(MAE)
        RF_score_teste.append(cscore)

        RF_mse_real.append(MSE_real)
        RF_rmse_real.append(RMSE_real)
        RF_mae_real.append(MAE_real)
        RF_score_real.append(cscore_real)

        # print dos valores preditos e reais para teste
        # teste4 = pd.DataFrame({"real": y_real, "predito": pred_real.tolist()})
        # print(teste4.head(20))

    # Finalizando Timer e Adicionando a Lista de Tempo do random forest
    fim = time.time()
    RF_tempo.append((fim - inicio))

    # Inicializando Timer para arvore de decisão
    inicio = time.time()

    # Loop para executar a validação cruzada da arvore de decisão
    for fold, (train_index, test_index) in enumerate(kf.split(X)):
        print(f"Fold {fold + 1}")

        # Separando casos de Treino e casos de Teste
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Pre-processando
        scaler = StandardScaler()
        scaler.fit(X_train)

        # Normalizando dados de Teste
        X_train_scaled = scaler.transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Normalizando dados Reais
        X_real_scaled = scaler.transform(X_real)

        # Treinando modelo de KNN baseado no X e y de Treino
        dt.fit(X_train_scaled, y_train)

        # Realizando Predições
        pred = dt.predict(X_test_scaled)
        ypred_real = dt.predict(X_real_scaled)

        # Validando casos de Teste
        MSE = mean_squared_error(y_test, pred, squared=True)
        RMSE = mean_squared_error(y_test, pred, squared=False)
        MAE = mean_absolute_error(y_test, pred)
        cscore = r2_score(y_test, pred)

        # Validando casos Reais
        MSE_real = mean_squared_error(y_real, ypred_real, squared=True)
        RMSE_real = mean_squared_error(y_real, ypred_real, squared=False)
        MAE_real = mean_absolute_error(y_real, ypred_real)
        cscore_real = r2_score(y_real, ypred_real)

        # Adicionando valores a lista criada anteriormente para poder pegar a média
        DT_mse_teste.append(MSE)
        DT_rmse_teste.append(RMSE)
        DT_mae_teste.append(MAE)
        DT_score_teste.append(cscore)

        DT_mse_real.append(MSE_real)
        DT_rmse_real.append(RMSE_real)
        DT_mae_real.append(MAE_real)
        DT_score_real.append(cscore_real)

        # print dos valores preditos e reais para teste
        # teste5 = pd.DataFrame({"real": y_real, "predito": pred_real.tolist()})
        # print(teste5.head(20))

    # Finalizando Timer e Adicionando a Lista de Tempo da Decision Tree
    fim = time.time()
    DT_tempo.append((fim - inicio))


Repetition 1
Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Fold 6
Fold 7
Fold 8
Fold 9
Fold 10
Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Fold 6
Fold 7
Fold 8
Fold 9
Fold 10
Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Fold 6
Fold 7
Fold 8
Fold 9
Fold 10
Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Fold 6
Fold 7
Fold 8
Fold 9
Fold 10
Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Fold 6
Fold 7
Fold 8
Fold 9
Fold 10


In [8]:
# Printando dados na tela
print("--------------Regressão-Linear------------------")
print("MSE Teste:", mean(RL_mse_teste))
print("RMSE Teste:", mean(RL_rmse_teste))
print("MAE Teste:", mean(RL_mae_teste))
print("R-scores Teste:", mean(RL_score_teste))

print("MSE Real:", mean(RL_mse_real))
print("RMSE Real:", mean(RL_rmse_real))
print("MAE Real:", mean(RL_mae_real))
print("R-scores Real:", mean(RL_score_real))

print("\n------------Rede-Neural-Artificial--------------")
print("MSE Teste:", mean(RNA_mse_teste))
print("RMSE Teste:", mean(RNA_rmse_teste))
print("MAE Teste:", mean(RNA_mae_teste))
print("R-scores Teste:", mean(RNA_score_teste))

print("MSE Real:", mean(RNA_mse_real))
print("RMSE Real:", mean(RNA_rmse_real))
print("MAE Real:", mean(RNA_mae_real))
print("R-scores Real:", mean(RNA_score_real))

print("\n--------------------KNN-k=7---------------------")
print("MSE Teste:", mean(KNN_mse_teste))
print("RMSE Teste:", mean(KNN_rmse_teste))
print("MAE Teste:", mean(KNN_mae_teste))
print("R-scores Teste:", mean(KNN_score_teste))

print("MSE Real:", mean(KNN_mse_real))
print("RMSE Real:", mean(KNN_rmse_real))
print("MAE Real:", mean(KNN_mae_real))
print("R-scores Real:", mean(KNN_score_real))

print("\n------------Random-Forest-----------------------")
print("MSE Teste:", mean(RF_mse_teste))
print("RMSE Teste:", mean(RF_rmse_teste))
print("MAE Teste:", mean(RF_mae_teste))
print("R-scores Teste:", mean(RF_score_teste))

print("MSE Real:", mean(RF_mse_real))
print("RMSE Real:", mean(RF_rmse_real))
print("MAE Real:", mean(RF_mae_real))
print("R-scores Real:", mean(RF_score_real))

print("-----------------Decision-Tree------------------")
print("MSE Teste:", mean(DT_mse_teste))
print("RMSE Teste:", mean(DT_rmse_teste))
print("MAE Teste:", mean(DT_mae_teste))
print("R-scores Teste:", mean(DT_score_teste))

print("MSE Real:", mean(DT_mse_real))
print("RMSE Real:", mean(DT_rmse_real))
print("MAE Real:", mean(DT_mae_real))
print("R-scores Real:", mean(DT_score_real))

print("\n--------------Tempo-dos-Algoritmos--------------")
print("Tempo Total RL (s): ", sum(RL_tempo))
print("Tempo Total RNA (s): ", sum(RNA_tempo))
print("Tempo Total KNN (s): ", sum(KNN_tempo))
print("Tempo Total RF (s): ", sum(RF_tempo))
print("Tempo Total DT (s): ", sum(DT_tempo))

--------------Regressão-Linear------------------
MSE Teste: 126.76982342690523
RMSE Teste: 9.972098953121254
MAE Teste: 2.771396281231473
R-scores Teste: 0.8585033941677498
MSE Real: 101.95227441937206
RMSE Real: 9.431661614371507
MAE Real: 3.1174800820773494
R-scores Real: 0.8733453378332425

------------Rede-Neural-Artificial--------------
MSE Teste: 50.53954538596308
RMSE Teste: 6.515788545619746
MAE Teste: 2.1600124901401188
R-scores Teste: 0.9184462361435971
MSE Real: 156.76424449691493
RMSE Real: 11.529164085807421
MAE Real: 3.1152830659186237
R-scores Real: 0.7735031432079349

--------------------KNN-k=7---------------------
MSE Teste: 44.62728433129497
RMSE Teste: 5.766535279719812
MAE Teste: 1.9582889597382518
R-scores Teste: 0.9498068294895321
MSE Real: 126.66219679945115
RMSE Real: 10.522665661826856
MAE Real: 3.1655357771717405
R-scores Real: 0.8197543082119072

------------Random-Forest-----------------------
MSE Teste: 63.918480915539384
RMSE Teste: 7.0463862378506565
MAE